In [1]:
import pandas as pd
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [8]:
train_df

,Category,IncidentGrade,EntityType,EvidenceRole,IpAddress,Url,AccountName,DayOfWeek,Month,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7
0,10,2,15,1,27,160396,453297,1,6,0,1,1,1,1,1,1
1,7,1,32,0,360606,160396,24887,4,6,1,1,1,0,1,1,1
2,10,1,31,1,360606,68652,453297,3,6,1,1,1,1,1,1,1
3,1,0,31,1,360606,13,453297,0,6,1,1,1,1,1,1,1
4,6,2,32,0,360606,160396,479,5,6,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9465492,10,2,6,1,360606,160396,453297,5,6,1,1,1,1,1,1,1
9465493,1,0,31,1,360606,164,453297,3,6,1,1,1,1,1,1,1
9465494,7,0,21,0,360606,160396,453297,1,6,1,1,1,0,0,1,1
9465495,9,0,15,1,465,160396,453297,0,5,1,1,1,1,1,1,1


In [9]:
test_df

,Category,IncidentGrade,EntityType,EvidenceRole,IpAddress,Url,AccountName,DayOfWeek,Month,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7
0,11,0,28,0,360606,160396,863,1,6,1,1,1,0,1,1,1
1,1,0,15,0,360606,160396,453297,0,6,1,2,0,1,0,1,1
2,11,0,23,1,360606,160396,453297,5,6,1,1,0,1,1,1,0
3,10,1,7,1,360606,160396,453297,2,6,1,1,1,1,1,1,1
4,5,0,28,0,360606,160396,136104,3,6,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4147987,10,0,28,0,360606,160396,14567,1,6,1,1,1,0,1,1,1
4147988,10,0,28,0,360606,160396,107151,1,6,1,1,1,0,1,1,1
4147989,1,1,12,1,1084,160396,453297,5,6,1,1,1,1,1,1,1
4147990,10,0,18,0,360606,160396,25428,1,6,1,1,1,0,1,1,1


In [10]:
train_df.isna().sum()

Category         0
IncidentGrade    0
EntityType       0
EvidenceRole     0
IpAddress        0
Url              0
AccountName      0
DayOfWeek        0
Month            0
Feature1         0
Feature2         0
Feature3         0
Feature4         0
Feature5         0
Feature6         0
Feature7         0
dtype: int64

In [11]:
test_df.isna().sum()

Category         0
IncidentGrade    0
EntityType       0
EvidenceRole     0
IpAddress        0
Url              0
AccountName      0
DayOfWeek        0
Month            0
Feature1         0
Feature2         0
Feature3         0
Feature4         0
Feature5         0
Feature6         0
Feature7         0
dtype: int64

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

# Assuming train_df is already defined
X = train_df.drop('IncidentGrade', axis=1)
y = train_df['IncidentGrade']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ColumnTransformer
numerical_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.columns)
    ])

# Fit and transform the training data
X_train_transformed = numerical_transformer.fit_transform(X_train)
X_test_transformed = numerical_transformer.transform(X_test)

# Apply undersampling
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_transformed, y_train)

# Initialize and train the classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_resampled, y_train_resampled)

# Perform cross-validation
cv_scores = cross_val_score(classifier, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean()}')

# Predict and evaluate
y_pred = classifier.predict(X_test_transformed)
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1', 'Class 2'])
print(report)

# Assuming test_df is already defined and has the same structure as train_df
X_test_df = test_df.drop('IncidentGrade', axis=1)
y_test_df = test_df['IncidentGrade']

# Transform the test data
X_test_df_transformed = numerical_transformer.transform(X_test_df)

# Predict using the classifier
y_pred_test_df = classifier.predict(X_test_df_transformed)

# Evaluate the predictions
report_test_df = classification_report(y_test_df, y_pred_test_df, target_names=['Class 0', 'Class 1', 'Class 2'])
print(report_test_df)

#137m 49.3s

Cross-validation scores: [0.71328868 0.71372437 0.71261516 0.71306081 0.71360824]
Mean cross-validation score: 0.7132594529747321
              precision    recall  f1-score   support

     Class 0       0.76      0.77      0.76    823087
     Class 1       0.58      0.66      0.62    406186
     Class 2       0.81      0.73      0.77    663827

    accuracy                           0.73   1893100
   macro avg       0.72      0.72      0.72   1893100
weighted avg       0.74      0.73      0.74   1893100

              precision    recall  f1-score   support

     Class 0       0.71      0.71      0.71   1752940
     Class 1       0.52      0.63      0.57    902698
     Class 2       0.82      0.70      0.75   1492354

    accuracy                           0.69   4147992
   macro avg       0.68      0.68      0.68   4147992
weighted avg       0.71      0.69      0.70   4147992

